<a href="https://colab.research.google.com/github/manuelboi/MLsec_project/blob/main/MLsec_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Models:
1. Fixing Data Augmentation to Improve Adversarial Robustness
2. Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness?
3. MMA Training: Direct Input Space Margin Maximization through Adversarial Training

# General imports

In [1]:
%%capture

# Various
import matplotlib.pyplot as plt
import time
import importlib.util
import re

# Pytorch
import torch
from torch.utils.data import DataLoader, Subset
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

# RobustBench
!pip install git+https://github.com/manuelboi/robustbench
from robustbench.utils import load_model
from robustbench.eval import benchmark
from robustbench.data import load_cifar10
from robustbench.model_zoo.enums import ThreatModel

# Smoothing
!git clone https://github.com/matteoturnu/smoothing.git
!conda create -n smoothing
!conda activate smoothing
!conda install pytorch torchvision cudatoolkit=10.0 -c pytorch
!conda install scipy pandas statsmodels matplotlib seaborn
!pip install setGPU
from smoothing.code.core import Smooth
from smoothing.code.train import train, test
from smoothing.code.train_utils import AverageMeter, accuracy

# Setup

In [2]:
%%capture

labels_dct = {0: "airplane", 1: "automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog", 6: "frog", 7: "horse", 8: "ship", 9: "truck"}
# use GPU if available, otherwise use CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Computing device used: ", device)

# Preparing trainset and testset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64
n_test_samples_benchmark = 100
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


# Define the number of samples you want to take
num_samples = 1000
# Create a subset of the dataset
indices = list(range(num_samples))
train_subset = Subset(trainset, indices)
test_subset = Subset(testset, indices)

# Preparing trainloader and testloader
trainloader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_subset, batch_size=batch_size, shuffle=False, num_workers=2)

'''
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
'''

# Load test samples for predict and certify
x_test, y_test = load_cifar10(n_test_samples_benchmark)
x_test, y_test = x_test.to(device), y_test.to(device)

# Setting various parameters
epochs = 3 # used for training
sigma = 0.5 # gaussian noise standard deviation
n_examples = 20 # to be perturbed by AutoAttack
eps_L2 = 0.5 # epsilon of the perturbancy for L2 norm
eps_Linf = 8/255 # epsilon of the perturbancy with Linf norm
version = 'custom'
attacks_to_run=['apgd-ce', 'apgd-dlr'] # Attacks to run on the models

# Functions

Model training

In [3]:
def train_model(trainloader, testloader, model, epochs, sigma, device):
  criterion = CrossEntropyLoss().to(device)
  optimizer = SGD(model.parameters())
  scheduler = StepLR(optimizer, step_size=30)

  for epoch in range(epochs):
    scheduler.step(epoch)
    before = time.time()
    train_loss, train_acc = train(trainloader, model, criterion, optimizer, epoch, sigma, device)
    test_loss, test_acc = test(testloader, model, criterion, epoch, sigma, device)
    after = time.time()

  return model

Smooth model performances on perturbed images

In [4]:
def certify(model, sigma, x_text, y_test, L):
  n_classes = 10
  alpha = 1 # (1 - alpha) is the confidence level (in this case is 0)
  n0 = 10 # number of samples for selection
  n = 20 # number of samples for estimation (certify) (too few samples but computation time is strongly affected with more)

  smooth_model = Smooth(model, n_classes, sigma)

  top_classes = list()
  radiuses = list()
  for x, y in zip(x_test, y_test):
    top_class = smooth_model.predict(x, n0, alpha, batch_size=n0, device=device,)
    top_class, radius = smooth_model.certify(x, n0, n, alpha, batch_size=n0, device=device, L=L)
    top_classes.append(top_class)
    radiuses.append(radius)

  top_classes = torch.tensor(top_classes, dtype=torch.float64).to(device)
  accuracy = torch.mean(top_classes == y_test, dtype=torch.float64)

  print("Top classes: ", top_classes)
  print("Y classes: ", y_test)
  print("Accuracy on smooth model: ", accuracy)

# Fixing Data Augmentation to Improve Adversarial Robustness (WideResNet-70-16)

L2

In [5]:
# Model loading
model_1_L2 = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='L2')
model_1_L2 = model_1_L2.to(device)
model_1_L2.eval()

# Model training
model_1_L2 = train_model(trainloader, testloader, model_1_L2, epochs, sigma, device)

# Model prediction and certification
certify(model_1_L2, sigma, x_test, y_test, 'L2')

# AutoAttack on model 1 with L2
benchmark(model_1_L2, threat_model=ThreatModel.L2, n_examples=n_examples, eps=eps_L2, batch_size=batch_size, device=device, version=version, attacks_to_run=attacks_to_run)

Downloading...
From (original): https://drive.google.com/uc?id=1JX82BDVBNO-Ffa2J37EuB8C-aFCbz708
From (redirected): https://drive.google.com/uc?id=1JX82BDVBNO-Ffa2J37EuB8C-aFCbz708&confirm=t&uuid=cfe4a3f6-b1bb-4ca3-84e7-3916c2d8b340
To: /content/models/cifar10/L2/Rebuffi2021Fixing_70_16_cutmix_extra.pt
100%|██████████| 1.07G/1.07G [00:19<00:00, 53.8MB/s]
/usr/local/lib/python3.10/dist-packages/robustbench/utils.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless 

Epoch: [0][0/16]	Time 3.605 (3.605)	Data 0.161 (0.161)	Loss 0.5563 (0.5563)	Acc@1 89.062 (89.062)	Acc@5 100.000 (100.000)
Epoch: [0][1/16]	Time 0.579 (2.092)	Data 0.014 (0.087)	Loss 0.5093 (0.5328)	Acc@1 87.500 (88.281)	Acc@5 98.438 (99.219)
Epoch: [0][2/16]	Time 1.688 (1.958)	Data 0.011 (0.062)	Loss 0.5026 (0.5227)	Acc@1 89.062 (88.542)	Acc@5 98.438 (98.958)
Epoch: [0][3/16]	Time 1.673 (1.887)	Data 0.003 (0.047)	Loss 0.4389 (0.5018)	Acc@1 92.188 (89.453)	Acc@5 100.000 (99.219)
Epoch: [0][4/16]	Time 1.675 (1.844)	Data 0.003 (0.038)	Loss 0.4663 (0.4947)	Acc@1 89.062 (89.375)	Acc@5 98.438 (99.062)
Epoch: [0][5/16]	Time 1.677 (1.816)	Data 0.003 (0.032)	Loss 0.3340 (0.4679)	Acc@1 93.750 (90.104)	Acc@5 100.000 (99.219)
Epoch: [0][6/16]	Time 1.678 (1.797)	Data 0.003 (0.028)	Loss 0.4582 (0.4665)	Acc@1 89.062 (89.955)	Acc@5 100.000 (99.330)
Epoch: [0][7/16]	Time 1.681 (1.782)	Data 0.003 (0.025)	Loss 0.3918 (0.4572)	Acc@1 96.875 (90.820)	Acc@5 98.438 (99.219)
Epoch: [0][8/16]	Time 1.687 (1.772)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/autoattack/checks.py", line 100, in check_dynamic
    sys.settrace(tracefunc)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/autoattack/checks.py", line 102, in check_dynamic
    sys.settrace(None)



initial accuracy: 60.00%
apgd-ce - 1/1 - 4 out of 12 successfully perturbed
robust accuracy after APGD-CE: 40.00% (total time 164.6 s)
apgd-dlr - 1/1 - 0 out of 8 successfully perturbed
robust accuracy after APGD-DLR: 40.00% (total time 326.9 s)
max L2 perturbation: 0.50000, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 40.00%
Adversarial accuracy: 40.00%


(0.6, 0.4)

Linf

In [6]:
# Model loading
model_1_Linf = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='Linf')
model_1_Linf = model_1_Linf.to(device)
model_1_Linf.eval()

# Model training
model_1_Linf = train_model(trainloader, testloader, model_1_Linf, epochs, sigma, device)

# Model prediction and certification
certify(model_1_Linf, sigma, x_test, y_test, 'Linf')

# AutoAttack on model 1 with Linf
benchmark(model_1_Linf, threat_model=ThreatModel.Linf, n_examples=n_examples, eps=eps_Linf, batch_size=batch_size, device=device, version=version, attacks_to_run=attacks_to_run)

Downloading...
From (original): https://drive.google.com/uc?id=1qKDTp6IJ1BUXZaRtbYuo_t0tuDl_4mLg
From (redirected): https://drive.google.com/uc?id=1qKDTp6IJ1BUXZaRtbYuo_t0tuDl_4mLg&confirm=t&uuid=919c7b17-957c-4129-8022-9ebede38e43c
To: /content/models/cifar10/Linf/Rebuffi2021Fixing_70_16_cutmix_extra.pt
100%|██████████| 1.07G/1.07G [00:23<00:00, 44.9MB/s]


Epoch: [0][0/16]	Time 0.770 (0.770)	Data 0.132 (0.132)	Loss 0.7303 (0.7303)	Acc@1 85.938 (85.938)	Acc@5 100.000 (100.000)
Epoch: [0][1/16]	Time 1.743 (1.257)	Data 0.004 (0.068)	Loss 0.6788 (0.7046)	Acc@1 89.062 (87.500)	Acc@5 98.438 (99.219)
Epoch: [0][2/16]	Time 1.735 (1.416)	Data 0.004 (0.046)	Loss 0.7634 (0.7242)	Acc@1 84.375 (86.458)	Acc@5 100.000 (99.479)
Epoch: [0][3/16]	Time 1.758 (1.502)	Data 0.002 (0.035)	Loss 0.7559 (0.7321)	Acc@1 81.250 (85.156)	Acc@5 100.000 (99.609)
Epoch: [0][4/16]	Time 1.770 (1.555)	Data 0.010 (0.030)	Loss 0.7610 (0.7379)	Acc@1 85.938 (85.312)	Acc@5 98.438 (99.375)
Epoch: [0][5/16]	Time 1.762 (1.590)	Data 0.003 (0.026)	Loss 0.7749 (0.7440)	Acc@1 81.250 (84.635)	Acc@5 98.438 (99.219)
Epoch: [0][6/16]	Time 1.769 (1.615)	Data 0.014 (0.024)	Loss 0.7323 (0.7424)	Acc@1 85.938 (84.821)	Acc@5 100.000 (99.330)
Epoch: [0][7/16]	Time 1.787 (1.637)	Data 0.003 (0.021)	Loss 0.6940 (0.7363)	Acc@1 85.938 (84.961)	Acc@5 98.438 (99.219)
Epoch: [0][8/16]	Time 1.794 (1.654)

(0.45, 0.25)

# Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness? (ResNet-18)

L2

In [7]:
# Model loading
model_2_L2 = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='L2')
model_2_L2 = model_2_L2.to(device)
model_2_L2.eval()

# Model training
model_2_L2 = train_model(trainloader, testloader, model_2_L2, epochs, sigma, device)

# Model prediction and certification
certify(model_2_L2, sigma, x_test, y_test, 'L2')

# AutoAttack on model 2 with L2
benchmark(model_2_L2, threat_model=ThreatModel.L2, n_examples=n_examples, eps=eps_L2, batch_size=batch_size, device=device, version=version, attacks_to_run=attacks_to_run)

Downloading...
From (original): https://drive.google.com/uc?id=1UviikNzpltVFsgMuqQ8YhpmvGczGRS4S
From (redirected): https://drive.google.com/uc?id=1UviikNzpltVFsgMuqQ8YhpmvGczGRS4S&confirm=t&uuid=3da971c8-0254-4ff4-a0d0-c741487d7e67
To: /content/models/cifar10/L2/Sehwag2021Proxy.pt
100%|██████████| 1.11G/1.11G [00:29<00:00, 38.2MB/s]


Epoch: [0][0/16]	Time 0.437 (0.437)	Data 0.162 (0.162)	Loss 0.2338 (0.2338)	Acc@1 93.750 (93.750)	Acc@5 100.000 (100.000)
Epoch: [0][1/16]	Time 0.355 (0.396)	Data 0.002 (0.082)	Loss 0.1599 (0.1969)	Acc@1 96.875 (95.312)	Acc@5 100.000 (100.000)
Epoch: [0][2/16]	Time 0.368 (0.387)	Data 0.003 (0.056)	Loss 0.2277 (0.2072)	Acc@1 92.188 (94.271)	Acc@5 100.000 (100.000)
Epoch: [0][3/16]	Time 0.377 (0.384)	Data 0.003 (0.042)	Loss 0.2850 (0.2266)	Acc@1 92.188 (93.750)	Acc@5 100.000 (100.000)
Epoch: [0][4/16]	Time 0.371 (0.382)	Data 0.003 (0.034)	Loss 0.1648 (0.2143)	Acc@1 95.312 (94.062)	Acc@5 100.000 (100.000)
Epoch: [0][5/16]	Time 0.371 (0.380)	Data 0.003 (0.029)	Loss 0.2147 (0.2143)	Acc@1 93.750 (94.010)	Acc@5 100.000 (100.000)
Epoch: [0][6/16]	Time 0.378 (0.380)	Data 0.004 (0.026)	Loss 0.1222 (0.2012)	Acc@1 96.875 (94.420)	Acc@5 100.000 (100.000)
Epoch: [0][7/16]	Time 0.374 (0.379)	Data 0.003 (0.023)	Loss 0.2738 (0.2103)	Acc@1 93.750 (94.336)	Acc@5 100.000 (100.000)
Epoch: [0][8/16]	Time 0.

(0.25, 0.15)

Linf

In [8]:
# Model loading
model_2_Linf = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='Linf')
model_2_Linf = model_2_Linf.to(device)
model_2_Linf.eval()

# Model training
model_2_Linf = train_model(trainloader, testloader, model_2_Linf, epochs, sigma, device)

# Model prediction and certification
certify(model_2_Linf, sigma, x_test, y_test, 'Linf')

# AutoAttack on model 2 with Linf
benchmark(model_2_Linf, threat_model=ThreatModel.Linf, n_examples=n_examples, eps=eps_Linf, batch_size=batch_size, device=device, version=version, attacks_to_run=attacks_to_run)

Downloading...
From (original): https://drive.google.com/uc?id=1QFA5fPMj2Qw4aYNG33PkFqiv_RTDWvzm
From (redirected): https://drive.google.com/uc?id=1QFA5fPMj2Qw4aYNG33PkFqiv_RTDWvzm&confirm=t&uuid=56a4d53f-91aa-4bf1-b826-ee7fda184205
To: /content/models/cifar10/Linf/Sehwag2021Proxy.pt
100%|██████████| 1.11G/1.11G [00:19<00:00, 56.5MB/s]


Epoch: [0][0/16]	Time 0.427 (0.427)	Data 0.214 (0.214)	Loss 0.2920 (0.2920)	Acc@1 95.312 (95.312)	Acc@5 100.000 (100.000)
Epoch: [0][1/16]	Time 0.365 (0.396)	Data 0.003 (0.108)	Loss 0.3638 (0.3279)	Acc@1 87.500 (91.406)	Acc@5 100.000 (100.000)
Epoch: [0][2/16]	Time 0.364 (0.385)	Data 0.009 (0.075)	Loss 0.3006 (0.3188)	Acc@1 92.188 (91.667)	Acc@5 100.000 (100.000)
Epoch: [0][3/16]	Time 0.379 (0.384)	Data 0.003 (0.057)	Loss 0.3908 (0.3368)	Acc@1 84.375 (89.844)	Acc@5 100.000 (100.000)
Epoch: [0][4/16]	Time 0.373 (0.382)	Data 0.011 (0.048)	Loss 0.3637 (0.3422)	Acc@1 89.062 (89.688)	Acc@5 96.875 (99.375)
Epoch: [0][5/16]	Time 0.360 (0.378)	Data 0.013 (0.042)	Loss 0.2927 (0.3339)	Acc@1 95.312 (90.625)	Acc@5 100.000 (99.479)
Epoch: [0][6/16]	Time 0.375 (0.377)	Data 0.003 (0.037)	Loss 0.2849 (0.3269)	Acc@1 92.188 (90.848)	Acc@5 100.000 (99.554)
Epoch: [0][7/16]	Time 0.371 (0.377)	Data 0.002 (0.032)	Loss 0.3642 (0.3316)	Acc@1 85.938 (90.234)	Acc@5 100.000 (99.609)
Epoch: [0][8/16]	Time 0.376 (

(0.3, 0.15)

# MMA Training: Direct Input Space Margin Maximization through Adversarial Training (WideResNet-28-4)

L2

In [9]:
# Model loading
model_3_L2 = load_model(model_name='Ding2020MMA', dataset='cifar10', threat_model='L2')
model_3_L2 = model_3_L2.to(device)
model_3_L2.eval()

# Model training
model_3_L2 = train_model(trainloader, testloader, model_3_L2, epochs, sigma, device)

# Model prediction and certification
certify(model_3_L2, sigma, x_test, y_test, 'L2')

# AutoAttack on model 3 with L2
benchmark(model_3_L2, threat_model=ThreatModel.L2, n_examples=n_examples, eps=eps_L2, batch_size=batch_size, device=device, version=version, attacks_to_run=attacks_to_run)

Downloading...
From: https://drive.google.com/uc?id=13wgY0Q_eor52ltZ0PkfJx5BCZ8cLM52E
To: /content/models/cifar10/L2/Ding2020MMA.pt
100%|██████████| 23.4M/23.4M [00:00<00:00, 101MB/s] 


Epoch: [0][0/16]	Time 0.217 (0.217)	Data 0.153 (0.153)	Loss 1.0921 (1.0921)	Acc@1 76.562 (76.562)	Acc@5 96.875 (96.875)
Epoch: [0][1/16]	Time 0.081 (0.149)	Data 0.004 (0.078)	Loss 0.9147 (1.0034)	Acc@1 82.812 (79.688)	Acc@5 96.875 (96.875)
Epoch: [0][2/16]	Time 0.091 (0.130)	Data 0.001 (0.052)	Loss 0.6653 (0.8907)	Acc@1 79.688 (79.688)	Acc@5 98.438 (97.396)
Epoch: [0][3/16]	Time 0.087 (0.119)	Data 0.002 (0.040)	Loss 0.8752 (0.8868)	Acc@1 76.562 (78.906)	Acc@5 100.000 (98.047)
Epoch: [0][4/16]	Time 0.070 (0.109)	Data 0.003 (0.032)	Loss 0.9947 (0.9084)	Acc@1 71.875 (77.500)	Acc@5 95.312 (97.500)
Epoch: [0][5/16]	Time 0.068 (0.102)	Data 0.002 (0.027)	Loss 0.7117 (0.8756)	Acc@1 79.688 (77.865)	Acc@5 98.438 (97.656)
Epoch: [0][6/16]	Time 0.071 (0.098)	Data 0.002 (0.024)	Loss 0.6950 (0.8498)	Acc@1 78.125 (77.902)	Acc@5 100.000 (97.991)
Epoch: [0][7/16]	Time 0.070 (0.094)	Data 0.010 (0.022)	Loss 1.2003 (0.8936)	Acc@1 71.875 (77.148)	Acc@5 100.000 (98.242)
Epoch: [0][8/16]	Time 0.065 (0.091)	D

(0.85, 0.3)

Linf

In [10]:
# Model loading
model_3_Linf = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='Linf')
model_3_Linf = model_3_Linf.to(device)
model_3_Linf.eval()

# Model training
model_3_Linf = train_model(trainloader, testloader, model_3_Linf, epochs, sigma, device)

# Model prediction and certification
certify(model_3_Linf, sigma, x_test, y_test, 'Linf')

# AutoAttack on model 3 with Linf
benchmark(model_3_Linf, threat_model=ThreatModel.Linf, n_examples=n_examples, eps=eps_Linf, batch_size=batch_size, device=device, version=version, attacks_to_run=attacks_to_run)

Epoch: [0][0/16]	Time 0.291 (0.291)	Data 0.148 (0.148)	Loss 0.2397 (0.2397)	Acc@1 95.312 (95.312)	Acc@5 100.000 (100.000)
Epoch: [0][1/16]	Time 0.374 (0.333)	Data 0.005 (0.077)	Loss 0.4240 (0.3318)	Acc@1 89.062 (92.188)	Acc@5 98.438 (99.219)
Epoch: [0][2/16]	Time 0.395 (0.354)	Data 0.001 (0.052)	Loss 0.2899 (0.3178)	Acc@1 96.875 (93.750)	Acc@5 100.000 (99.479)
Epoch: [0][3/16]	Time 0.383 (0.361)	Data 0.003 (0.039)	Loss 0.3915 (0.3363)	Acc@1 89.062 (92.578)	Acc@5 98.438 (99.219)
Epoch: [0][4/16]	Time 0.375 (0.364)	Data 0.001 (0.032)	Loss 0.3671 (0.3424)	Acc@1 84.375 (90.938)	Acc@5 100.000 (99.375)
Epoch: [0][5/16]	Time 0.388 (0.368)	Data 0.003 (0.027)	Loss 0.3238 (0.3393)	Acc@1 89.062 (90.625)	Acc@5 100.000 (99.479)
Epoch: [0][6/16]	Time 0.378 (0.369)	Data 0.003 (0.023)	Loss 0.2725 (0.3298)	Acc@1 90.625 (90.625)	Acc@5 100.000 (99.554)
Epoch: [0][7/16]	Time 0.376 (0.370)	Data 0.003 (0.021)	Loss 0.3198 (0.3285)	Acc@1 90.625 (90.625)	Acc@5 100.000 (99.609)
Epoch: [0][8/16]	Time 0.386 (0.37

(0.35, 0.2)